In [7]:
from db import call_ignition, call_db_json, insert_many_with_df, update_ignition
from queries import hk_flag, get_set_flag
from util_func import get_mondays, BlockKanbanStorage, BlockCoverage
from datetime import date, timedelta
import pandas as pd
import numpy as np
import bisect
block_kanban = BlockKanbanStorage()
block_coverage = BlockCoverage()

In [34]:
def check_date_group(dt, mondays):
    index = bisect.bisect_right(mondays, dt)
    if index == len(mondays):
        return 25
    else:
        return mondays[index - 1]
mondays = get_mondays(23)
mondays = [i.date() for i in mondays]

sql = """
SELECT * from manufacturing_orders
    WHERE product_family = 'HSR'
        AND order_customer_name = 'HAAS AUTOMATION'
        AND product_model = '45'
        AND product_variant = 'C'
        AND product_block_count > 0
        AND order_status = 10
        """
sql2 = """
    SELECT item_number, item_description, product_family, product_model, product_variant, product_block_count, warehouse_location, quantity, virtual_location  from current_wip_inventory
        WHERE item_description like 'SHS30%'
            AND item_description like '%BLOCK%'
            AND virtual_location = 'GY4'
"""
df = pd.DataFrame(call_db_json(sql2))
# df["week"] = df["order_scheduled_due"].apply(
#     check_date_group, args=[mondays]
# )


In [35]:
df.groupby(['product_variant', 'warehouse_location']).sum('quantity')

product_block_count  quantity
product_variant warehouse_location                               
C               GR01                                  1      17.0
                GR02                                  1       5.0
LC              GR01                                  1      27.0
LR              GR01                                  1      47.0
LV              GR01                                  1       0.0
                GR02                                  1     243.0
                GYSET4                                1       2.0
R               GR01                                  1       1.0
V               GR01                                  1      13.0
                GR02                                  1      63.0
                GYHAAS                                1     138.0

In [47]:
sql3 = """
    SELECT item_number, item_description, product_family, product_model, product_variant, product_block_count, warehouse_location, quantity, virtual_location  from current_wip_inventory
        WHERE product_family = 'SHS30'            
    """
df3 = pd.DataFrame(call_db_json(sql3))
df3


""


In [42]:
df3

     order_number customer_job_number reference_number      item_number  \
0         M909604                None           GYHAAS  301E510191826K0   
1         M917291                None           GYHAAS  301E510191826K0   
2         M917302                None           GYHAAS  301E510191826K0   
3         M917303                None           GYHAAS  301E510191826K0   
4         M098354                None           GYHAAS  301E510191826K0   
...           ...                 ...              ...              ...   
3309      M227487                None           GYHAAS  301E510191826K0   
3310      M227488                None           GYHAAS  301E510191826K0   
3311      M225543                None           GYHAAS  301E510191826K0   
3312      M225564                None           GYHAAS  301E510191826K0   
3313      M230899                None           GYHAAS  301E510191826K0   

            item_description product_family product_model product_variant  \
0     HSR45C1SSC0S(GP)